In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import multivariate_normal as ml
import plotly.graph_objects as go
from ipywidgets import interact
from ipywidgets import widgets

In [3]:
def rebalance(mu1 = 0, mu2= 0,vol1 = 0.1, vol2= 0.1, corr =0, moneyA= 500, moneyB=500, size=100 ):
    
    
    var1=vol1**2
    var2=vol2**2
    corr = np.array([1,corr,corr,1]).reshape(2,2)
    #cov_matrix = np.array([var1,cov,cov,var2]).reshape(2,2)
    sigma = np.diag([var1,var2])
    cov_matrix = np.matmul(np.matmul((sigma),corr),(sigma))
    #corr = np.matmul(np.linalg.inv(sigma),cov_matrix)
    
    reb = ml([mu1,mu2],cov_matrix, size=size)
    
    shares_a = 20*(1+reb[:,0])
    shares_b = 20*(1+reb[:,1])
    rets = pd.DataFrame({"SharesA":shares_a, "SharesB": shares_b})
    
    SPY_shares= moneyA/rets["SharesA"].iloc[0]
    TLT_shares= moneyB/rets["SharesB"].iloc[0]

    equityWeight = moneyA/(moneyA+moneyB)
    bondsWeight = moneyB/(moneyA+moneyB)

    newmktSP=[]
    newmktTLT=[]

    newSharesSP= SPY_shares
    newSharesTLT= TLT_shares

    for i, rows in rets.iterrows():
        
        mktvalSP= (newSharesSP)*(rets["SharesA"][i])
        mktvalTLT= (newSharesTLT)*(rets["SharesB"][i])
    
        weightSP = mktvalSP/(mktvalSP+mktvalTLT)
        weightTLT = mktvalTLT/(mktvalSP+mktvalTLT)
    

        buySellSPY = (weightSP - equityWeight)*(mktvalSP+mktvalTLT)
        buySellTLT = (weightTLT - bondsWeight)*(mktvalSP+mktvalTLT)
    
        newmktValSP = mktvalSP-buySellSPY
        newmktValTLT = mktvalTLT-buySellTLT
    
    
        newSharesSP = newmktValSP/rets["SharesA"][i]
        newSharesTLT = newmktValTLT/rets["SharesB"][i]
    
    
        #addSP = newSharesSP - (sharesSP)
        #SPADD = addSP + newSharesSP
    
    
        addTLT =  newSharesTLT 
    
    
        #newmktSP.append(mktvalSP)
        newmktSP.append(newmktValSP)
        #newmktTLT.append(mktvalTLT) 
        newmktTLT.append(newmktValTLT)
    
    rebalancedTLT = pd.DataFrame(newmktTLT,columns=["TLT"])
    rebalancedSPY = pd.DataFrame(newmktSP,columns=["SPY"])
    reb = rebalancedSPY.join(rebalancedTLT)
    reb["Total"] = reb["SPY"]+reb["TLT"]
    
    non_reb = pd.DataFrame({"A":rets["SharesA"]*moneyA, "B":rets["SharesB"]*moneyB})
    non_reb["Non_Total"] = non_reb["A"]+non_reb["B"]
    
    portfolios = pd.concat([reb["Total"],non_reb["Non_Total"]],axis=1)
    
    data = (1+portfolios.pct_change().dropna()).cumprod()
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=data["Total"],
                    mode='lines',
                    name='Rebalanced', line_color="maroon"))
    fig.add_trace(go.Scatter(y=data["Non_Total"],
                    mode='lines',
                    name='Not Rebalanced', line_color="grey"))
    fig.update_layout(
    autosize=False,
    width=1500,
    height=500,)
    

    return fig.show()




In [4]:
interact(rebalance, mu1=(0, 0.8, 0.05),mu2=(0, 0.8, 0.05),vol1 = (0, 0.8, 0.05),vol2 = (0, 0.8, 0.05), corr=(-1,1,0.1), moneyA=(500,10000, 100),moneyB=(500,10000, 100),
        size=(1,10000, 100));

interactive(children=(FloatSlider(value=0.0, description='mu1', max=0.8, step=0.05), FloatSlider(value=0.0, de…